In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [5]:
def vector_field(grayImage,labelled_img, label_frequency,blur_sigma = 1,blur_size =7,threshold = 0.5):
    h = grayImage.shape[0]
    w = grayImage.shape[1]

    out = cv2.GaussianBlur(grayImage,(blur_size,blur_size),blur_sigma)
    outx = cv2.Sobel(out,cv2.CV_64F,1,0,ksize=3)
    outy = cv2.Sobel(out,cv2.CV_64F,0,1,ksize=3)
    rotate_out = np.concatenate((outx.reshape(h,w,1),np.array(-1*outy).reshape(h,w,1)),axis=2)/255
    bool_array = (rotate_out[:,:,1]<0).astype(int)
    replicate_bool = np.concatenate((bool_array.reshape(h,w,1),bool_array.reshape(h,w,1)),axis = 2)
    
    rotate_out = rotate_out*(1 - 2*replicate_bool)
    mean_vectors = np.zeros((len(label_frequency),2))
    variance_vectors = np.zeros((len(label_frequency),1))
    for i in range(h):
        for j in range(w):
            mean_vectors[labelled_img[i,j],:] += rotate_out[i,j,:] #np.reshape(rotate_out[i,j,:],1,2)
    
    for i in range(len(label_frequency)):
        mean_vectors[i,:] /= label_frequency[i]
    
    for i in range(h):
        for j in range(w):
            variance_vectors[labelled_img[i,j]] += np.sum((rotate_out[i,j,:] - mean_vectors[labelled_img[i,j],:])**2)

    for i in range(len(label_frequency)):
        variance_vectors[i] /= label_frequency[i]
    
    for i in range(h):
        for j in range(w):
            if variance_vectors[labelled_img[i,j]] <= threshold:
                rotate_out[i,j,:] = mean_vectors[labelled_img[i,j],:]
    
    sum_vals = (np.sum(rotate_out*2,axis = 2))*0.5 + np.exp(-2)
    replicate = np.concatenate((sum_vals.reshape(h,w,1),sum_vals.reshape(h,w,1)),axis = 2)
    rotate_out = rotate_out / replicate

    return rotate_out